In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/czech-to-english/ces.txt
/kaggle/input/czech-to-english/_about.txt


In [2]:
english_texts=[]
czech_texts=[]
english_character=[]
czech_character=[]
with open("/kaggle/input/czech-to-english/ces.txt", "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[:20000]:
    english_text,czech_text,_=line.split("\t")
    english_texts.append(english_text)
    czech_text = "\t" + czech_text + "\n"
    czech_texts.append(czech_text)

In [3]:
for i in english_texts:
    for c in i:
        if c not in english_character:
            english_character.append(c)
            english_character.sort()


In [4]:
for j in czech_texts:
    for c in j:
        if c not in czech_character:
            czech_character.append(c)
            czech_character.sort()

In [5]:
english_d={}
for i in range(len(english_character)):
    english_d[english_character[i]]=i

In [6]:
czech_d={}
for i in range(len(czech_character)):
    czech_d[czech_character[i]]=i

In [7]:
max_encoder_seq_length=0
for i in english_texts:
    if len(i)>max_encoder_seq_length:
        max_encoder_seq_length=len(i)

In [8]:
max_decoder_seq_length=0
for i in czech_texts:
    if len(i)>max_decoder_seq_length:
        max_decoder_seq_length=len(i)

In [9]:
encoder_input_data=[]
for bb in range(20000):
    a=[]
    b=[]
    c=[]
    k=len(english_texts[bb])
    m=0
    while m<k:
        for char in english_texts[bb][m]:
                           for i in range(len(english_character)):
                                            if english_d[char]==i:
                                                a.append(1)
                                            else:
                                                a.append(0)
       
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
    while m<max_encoder_seq_length:
        for i in range(len(english_character)):
            if i==0:
                a.append(1)
            else:
                a.append(0)
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
    encoder_input_data.append(c)

In [10]:
encoder_input_data=np.array(encoder_input_data)

In [11]:
encoder_input_data.shape


(20000, 48, 75)

In [12]:
decoder_input_data=[]
for bb in range(20000):
    a=[]
    b=[]
    c=[]
    k=len(czech_texts[bb])
    m=0
    while m<k:
        for char in czech_texts[bb][m]:
                           for i in range(len(czech_character)):
                                            if czech_d[char]==i:
                                                a.append(1)
                                            else:
                                                a.append(0)
       
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
    while m<max_decoder_seq_length:
        for i in range(len(czech_character)):
            if i==0:
                a.append(1)
            else:
                a.append(0)
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
    decoder_input_data.append(c)

In [13]:
decoder_input_data=np.array(decoder_input_data)

In [14]:
decoder_input_data.shape

(20000, 71, 100)

In [15]:
decoder_target_data=[]
for bb in range(20000):
    a=[]
    b=[]
    c=[]
    k=len(czech_texts[bb])
    m=1
    while m<k:
        for char in czech_texts[bb][m]:
                           for i in range(len(czech_character)):
                                            if czech_d[char]==i:
                                                a.append(1)
                                            else:
                                                a.append(0)
       
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
    m=m-1
    while m<max_decoder_seq_length:
        for i in range(len(czech_character)):
            if i==0:
                a.append(1)
            else:
                a.append(0)
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
    decoder_target_data.append(c)

In [16]:
decoder_target_data=np.array(decoder_target_data)

In [17]:
decoder_target_data.shape

(20000, 71, 100)

In [18]:
batch_size = 64  # Batch size for training.
epochs = 100 # Number of epochs to train for.
latent_dim = 256 

In [19]:
from keras.layers import Dense
from keras.layers import LSTM,Input
from keras.models import Model

In [20]:
encoder_inputs = Input(shape=(None,len(english_character)))
encoder = LSTM(latent_dim,dropout=0.2,return_sequences=True,return_state=True)
encoder_outputs_1, state_h_1, state_c_1 = encoder(encoder_inputs)
encoder = LSTM(latent_dim,dropout=0.2,return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_outputs_1)
encoder_states = [state_h_1,state_c_1,state_h, state_c]

In [21]:
decoder_inputs = Input(shape=(None, len(czech_character)))
decoder_lstm = LSTM(latent_dim,return_sequences=True,dropout=0.2,return_state=True)
decoder_outputs_1, _, _ = decoder_lstm(decoder_inputs, initial_state=[state_h_1,state_c_1])
decoder_lstm_1 = LSTM(latent_dim, return_sequences=True,dropout=0.2,return_state=True)
decoder_outputs, _, _ = decoder_lstm_1(decoder_outputs_1, initial_state=[state_h,state_c])
decoder_dense = Dense(len(czech_character), activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)
model=Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [22]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 75)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 100)]  0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, None, 256),  339968      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  365568      input_2[0][0]                    
                                                                 lstm[0][1]            

In [23]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)

Epoch 1/100
250/250 [==============================] - 11s 43ms/step - loss: 1.1265 - accuracy: 0.7190 - val_loss: 1.4527 - val_accuracy: 0.6166
Epoch 2/100
250/250 [==============================] - 9s 36ms/step - loss: 0.8535 - accuracy: 0.7634 - val_loss: 1.2174 - val_accuracy: 0.6517
Epoch 3/100
250/250 [==============================] - 9s 37ms/step - loss: 0.7764 - accuracy: 0.7796 - val_loss: 1.1211 - val_accuracy: 0.6757
Epoch 4/100
250/250 [==============================] - 9s 37ms/step - loss: 0.7324 - accuracy: 0.7904 - val_loss: 1.0628 - val_accuracy: 0.6912
Epoch 5/100
250/250 [==============================] - 9s 36ms/step - loss: 0.6977 - accuracy: 0.7999 - val_loss: 0.9994 - val_accuracy: 0.7112
Epoch 6/100
250/250 [==============================] - 9s 36ms/step - loss: 0.6707 - accuracy: 0.8070 - val_loss: 0.9595 - val_accuracy: 0.7209
Epoch 7/100
250/250 [==============================] - 9s 37ms/step - loss: 0.6498 - accuracy: 0.8125 - val_loss: 0.9290 - val_accuracy

In [24]:
model.save("engtoczech.h5")

In [25]:
import keras

In [26]:
model = keras.models.load_model("engtoczech.h5")
encoder_inputs = model.input[0]  
encoder_outputs_1, state_h_enc_1, state_c_enc_1 = model.layers[2].output 
encoder_outputs, state_h_enc, state_c_enc = model.layers[4].output 
encoder_states = [state_h_enc_1, state_c_enc_1,state_h_enc, state_c_enc]
encoder_model_1 = keras.Model(encoder_inputs, encoder_states)

In [27]:
decoder_inputs = model.input[1]  
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_3")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_4")
decoder_state_input_h1 = Input(shape=(latent_dim,),name="input_5")
decoder_state_input_c1 = Input(shape=(latent_dim,),name="input_6")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c,decoder_state_input_h1,decoder_state_input_c1]
decoder_lstm = model.layers[3]
dec_o, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs[:2])
decoder_lstm_1=model.layers[5]
dec_o_1, state_h1, state_c1 = decoder_lstm_1(
    dec_o, initial_state=decoder_states_inputs[-2:])
decoder_states = [state_h,state_c,state_h1,state_c1]
decoder_dense = model.layers[6]
decoder_outputs = decoder_dense(dec_o_1)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

In [28]:
reverse_input_char_index ={}
for i in range(len(english_character)):
    reverse_input_char_index[i]=english_character[i]
reverse_target_char_index ={}
for i in range(len(czech_character)):
    reverse_target_char_index[i]=czech_character[i]


In [29]:
def decode_sequence(input_seq):
    states_value=encoder_model_1.predict(input_seq)
    target_seq = np.zeros((1, 1, len(czech_character)))
    target_seq[0, 0, czech_d["\t"]] = 1.0
    flag=0
    sent=""
    while flag==0:
        output_tokens, h, c,h1,c1 = decoder_model.predict([target_seq] + states_value)
        sample = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sample]
        sent+=sampled_char
        if sampled_char == "\n" or len(sent) > max_decoder_seq_length:
            flag=1
        target_seq = np.zeros((1, 1, len(czech_character)))
        target_seq[0, 0,sample] = 1.0
        states_value = [h, c,h1,c1]
    return sent


In [30]:
english='Who is?'

In [31]:
k=len(english)
m=0
a=[]
b=[]
c=[]
inpu=[]
while m<k:
    for char in english[m]:
        for i in range(len(english_character)):
            if english_d[char]==i:
                a.append(1)
            else:
                a.append(0)
    for kp in a:
        b.append(kp)
    c.append(b)
    b=[]
    a=[]
    m=m+1
while m<max_encoder_seq_length:
        for i in range(len(english_character)):
            if i==0:
                a.append(1)
            else:
                a.append(0)
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
inpu.append(c)

In [32]:
inpu=np.array(inpu)

In [33]:
inpu.shape

(1, 48, 75)

In [34]:
d=decode_sequence(inpu)

In [35]:
print(d)

Kdo je tvrd?

